In [ ]:
!pip install facenet-pytorch
!pip install torch torchvision


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 61.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 752.8/755.5 MB 153.5 MB/s eta 0:00:0100:01

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import NASNetLarge
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.nasnet import preprocess_input
import matplotlib.pyplot as plt


In [ ]:
# Configuration
IMG_HEIGHT, IMG_WIDTH = 331, 331
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.0001
MOMENTUM = 0.9
L2_REGULARIZATION = 0.001
MODEL_SAVE_PATH = "nasnet_large_emotion_model.h5"
DATASET_DIR = "/kaggle/input/affectnet-cleaned"

In [ ]:
# Data Augmentation
def create_data_generators(dataset_dir):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        validation_split=0.2  # 80% training, 20% validation
    )

    train_gen = datagen.flow_from_directory(
        os.path.join(dataset_dir, "Train"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        subset="training"
    )

    val_gen = datagen.flow_from_directory(
        os.path.join(dataset_dir, "Train"),
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        subset="validation"
    )

    return train_gen, val_gen

In [ ]:
# Build NasNet Large Model with Correct Metrics
def build_nasnet_large(num_classes):
    base_model = NASNetLarge(weights="imagenet", include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation="softmax", kernel_regularizer=tf.keras.regularizers.l2(L2_REGULARIZATION))(x)

    model = Model(inputs=base_model.input, outputs=output)

    # Freeze the base model for initial training
    base_model.trainable = False

    # Compile the model with accuracy as a metric
    model.compile(
        optimizer=SGD(learning_rate=LEARNING_RATE, momentum=MOMENTUM),
        loss="categorical_crossentropy",
        metrics=["accuracy"]  # Removed 'loss'
    )
    return model


In [ ]:
# Plot Training History
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.title("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Loss")
    plt.legend()

    plt.show()

In [ ]:
train_gen, val_gen = create_data_generators(DATASET_DIR)

In [ ]:
# Build the NasNetLarge model
model = build_nasnet_large(train_gen.num_classes)


In [ ]:
# Display the model summary
print(model.summary())

In [ ]:
# Train the model
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    verbose=2
)

In [ ]:
# Save the model
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


In [ ]:
# Plot training history
plot_training_history(history)

In [ ]:
# Fine-tune the base model (optional)
print("Fine-tuning the base model...")
model.layers[0].trainable = True  # Unfreeze the base model
model.compile(
    optimizer=SGD(learning_rate=LEARNING_RATE / 10, momentum=MOMENTUM),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS // 2,
    verbose=1
)

In [ ]:
# Save fine-tuned model
model.save("nasnet_large_finetuned.h5")
print("Fine-tuned model saved!")